In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy as np
import os

from torchdiffeq import  odeint
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
Batch_Size = 32
data_dir = './data'
os.makedirs(data_dir, exist_ok=True) # 데이터 폴더 생성

INPUT_DIM = 28 * 28 
TIME_T = 1.0

# ----------------------------------------------------------------------
# 2. FFJORD 최적화 전처리 정의 (핵심 수정 부분)
# ----------------------------------------------------------------------
# MNIST 데이터의 평균(Mean)과 표준편차(Std)는 0.5를 사용하여 [-1, 1] 범위로 표준화합니다.
class AddNoise(object):
    def __call__(self, tensor):
        # 0 ~ 1/256 사이의 균등 분포 노이즈 추가 인덱스는 연속적이여야 한다
        return tensor + torch.rand_like(tensor) / 256.0
def logit_transform(x, alpha=1e-6):
    x = x.clamp(min=alpha, max=1-alpha)
    return torch.log(x / (1 - x))

transform = transforms.Compose([
    transforms.ToTensor(), 
    AddNoise(),       # [0, 1] 범위로 변환
    transforms.Lambda(lambda x: logit_transform(x))
])
# ----------------------------------------------------------------------
# 3. 데이터 로더 정의 (수정된 transform 적용)
# ----------------------------------------------------------------------
train_dataset = datasets.MNIST(data_dir, train=True, download=True, transform=transform)
test_dataset = datasets.MNIST(data_dir, train=False, download=True, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=Batch_Size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=Batch_Size, shuffle=False)

In [2]:
#입력으로 f(x,t)와 y, t를 받아 허킨슨 추정량을 계산해주는 함수
#여러번 반복하여 평균을 내어 정확도를 높임
def Huchinson_estimator(forwardzt, y,  num_epsilons):
    Batch_Size=y.shape[0]
    grad_avg=torch.zeros(y.shape[0],1).to(device).to(y.dtype)
    for i in range(num_epsilons):
        epsilon=torch.randn_like(y).to(device).to(y.dtype)
        grad = torch.autograd.grad(forwardzt, y, grad_outputs=epsilon, create_graph=True)[0]
        trace_estimate = torch.sum(grad*epsilon, dim=1, keepdim=True)
        grad_avg += trace_estimate
    grad_avg /= num_epsilons
    return grad_avg

In [3]:
#t까지 ode solve 해주고 z(t)와 trace 반환. Huchinson_estimator 사용
class ODEFunc(nn.Module):
    def __init__(self, dim):
        super(ODEFunc, self).__init__()
        self.net=nn.Sequential(
            nn.Linear(dim+1, 128),
            nn.Softplus(),
            nn.Linear(128,128),
            nn.Softplus(),
            nn.Linear(128, dim),
        ) 
        self.dims = dim

    def forward1(self, t, x):
        # t: scalar tensor on GPU
        # x: (B, dim)
        t_tensor = t.expand(x.shape[0], 1)  # repeat for batch
        xt = torch.cat([x, t_tensor], dim=1)
        return self.net(xt)
    def forward(self, t, x):
        z, logpz_T = x

        # force t to GPU
        if not torch.is_tensor(t):
            t = torch.tensor([t], device=z.device, dtype=z.dtype)
        else:
            t = t.to(z.device).to(z.dtype)

        z_flat = z.view(z.size(0), -1)

        forwardzt = self.forward1(t, z_flat)
        trace = Huchinson_estimator(forwardzt, z_flat, num_epsilons=2)

        return forwardzt, -trace

In [4]:
#전체 신경망
class CNF(nn.Module):
    def __init__(self, ode_func, T=0.5):
        super().__init__()
        self.ode_func = ode_func
        self.time_t = T
        self.rtol=1e-5
        self.atol=1e-5
    #순방향 적분 생성과정
    def forward(self, z0, logpz=None):
        if logpz is None:
            logpz = torch.zeros(z0.shape[0],1).to(z0.device)
        time_tensor=torch.tensor([0.0, self.time_t], dtype=torch.float).to(z0.device)
        state_t = odeint(
            self.ode_func,
            (z0, logpz),
            time_tensor,
            method='rk4',
            rtol=self.rtol,
            atol=self.atol,
            options={}
        )
        zT=state_t[0][-1]
        logpz_T=state_t[1][-1]

        return zT, logpz_T
    #역방향 적분 학습과정
    def inverse(self, zT, logpz_T=None):
        zT = zT.requires_grad_(True)
        if logpz_T is None:
            logpz_T = torch.zeros(zT.shape[0],1).to(zT.device)
        time_tensor=torch.tensor([self.time_t, 0.0],dtype=torch.float).to(zT.device)
        state_t = odeint( 
            #위에 구현한 ODEFunc 사용           
            self.ode_func,
            # ODE의 초기 상태 (zT, logpz_T(trace에 해당))
            (zT, logpz_T),
            #시간 역방향으로 적분
            time_tensor,
            method='rk4',
            rtol=self.rtol,
            atol=self.atol,
            options={}
        )
        #각각 forwardzt와 trace가 적분된 결과 반환
        z0 = state_t[0][-1]      # 최종 상태 z0 (t=0, 노이즈)
        logpz_0 = state_t[1][-1] # 최종 로그 확률 보조 항 (log p(x) 계산에 사용)
        
        return z0, logpz_0

In [5]:
#표준 정규분포의 로그 확률 밀도 함수 계산
def get_logpz(z):
    logpz = -0.5 * torch.sum(z**2, dim=1, keepdim=True) - 0.5 * z.shape[1] * torch.tensor(np.log(2 * np.pi)).to(device)
    return logpz

In [6]:
def train (epoch, model, optimizer, train_loader, device, INPUT_DIM):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        data = data.view(data.shape[0], INPUT_DIM)


        #기울기 초기화
        optimizer.zero_grad()
        #모델의 역방향 적분 수행(데이타분포->표준정규분포) 이 z0이 나올확률을 최대화 해야한다.
        data = data.view(data.shape[0], INPUT_DIM)
        z0, logpz_0 = model.inverse(data)
        #z0에 대한 표준 정규분포의 로그 확률 밀도 함수 계산
        logpz = get_logpz(z0.view(z0.size(0), -1))
        #log p(x)= logp(z0) + log|det(dz0/dx)|(즉 -trace의 적분값)
        logpx = logpz + logpz_0
        #배치안 전체의 loss함수 평균 계산
        loss = -torch.mean(logpx)
        #역전파
        loss.backward()
        optimizer.step()
        train_loss += loss.item()
        if batch_idx % 100 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                (100. * batch_idx)/len(train_loader),
                loss.item()))
    avg_loss = train_loss / len(train_loader)
    print(f'Epoch: {epoch} Average loss: {avg_loss:.6f}')
    return avg_loss



In [7]:
def test(epoch, model, test_loader, device, INPUT_DIM):
    model.eval() # 평가 모드 설정
    test_loss = 0
    
    for data, _ in test_loader:
        data = data.to(device)
        data = data.view(data.shape[0], INPUT_DIM)
        data = data.view(data.shape[0], INPUT_DIM)
            # 1. Log-Likelihood 계산을 위해 역변환(Inverse)을 수행해야 함
        z0, logpz_0 = model.inverse(data)
            
            # 2. Log-Likelihood 계산
        logpz = get_logpz(z0.view(z0.size(0), -1))
        logpx = logpz + logpz_0
        loss = -torch.mean(logpx)

            # 3. 손실 누적 (한 에포크 모든 배치에 대해)
        test_loss += loss.item()
            
            
    # 배치 별 손실의 평균 계산
    avg_loss = test_loss / len(test_loader)
    print(f'====> Test set: Average loss: {avg_loss:.6f}')
    return avg_loss # 최종 손실 반환

In [8]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import os

# ----------------------------------------------------------------------
# 1. 하이퍼파라미터 및 환경 설정
# ----------------------------------------------------------------------
NUM_EPOCHS = 50
BATCH_SIZE = 32
INPUT_DIM = 28 * 28  # MNIST 이미지 평탄화 크기
TIME_T = 0.5         # ODE 적분 시간 T
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
os.makedirs('./logs', exist_ok=True) # 로그 폴더 생성

# ----------------------------------------------------------------------

train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# ----------------------------------------------------------------------
# 3. 모델 및 옵티마이저 초기화
# ----------------------------------------------------------------------
# (고객님의 ODEFunc와 CNF 클래스가 이전에 정의되어 있다고 가정합니다.)
# ODEFunc는 FFJORD의 f(z, t)를 정의합니다.
ode_func = ODEFunc(dim=INPUT_DIM).to(device)
ode_func = ode_func.float()
# CNF는 ODEFunc를 받아 odeint 솔버를 통해 흐름을 관리합니다.
model = CNF(ode_func, T=TIME_T).to(device)
model = model.float()

# Adam 옵티마이저 사용
optimizer = optim.Adam(model.parameters(), lr=5e-4)

# ----------------------------------------------------------------------
# 4. 메인 학습 및 테스트 루프
# ----------------------------------------------------------------------
train_history = []
test_history = []
best_test_loss = float('inf')
print(f"--- FFJORD Training Started on {device} ---")

for epoch in range(1, NUM_EPOCHS + 1):
    
    # [훈련] train 함수 호출
    train_loss = train(epoch, model, optimizer, train_loader, device, INPUT_DIM)
    train_history.append(train_loss)
    
    # [평가] test 함수 호출
    test_loss = test(epoch, model, test_loader, device, INPUT_DIM)
    test_history.append(test_loss)
    if test_loss < best_test_loss:
        best_test_loss = test_loss
        torch.save({
            'model_state_dict': model.state_dict(),
            'test_loss': best_test_loss,
        }, './logs/ffjord_best_NORM.pthp') # <-- 안정적인 모델 파일 저장
        print(f"✅ Loss 개선 확인: {best_test_loss:.4f}로 모델 저장 완료.")

print("--- Training Finished ---")
    # [모델 저장] (선택 사항)
    # if test_loss == min(test_history):
    #     torch.save(model.state_dict(), './logs/ffjord_best.pth')

--- FFJORD Training Started on cuda ---
Train Epoch: 1 [0/60000 (0%)]	Loss: 16390.597656


c:\Users\harryjo\Desktop\hwiwon ffjord\venv313\Lib\site-packages\torch\autograd\graph.py:865: UserWarning: Attempting to run cuBLAS, but there was no current CUDA context! Attempting to set the primary context... (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\pytorch\aten\src\ATen\cuda\CublasHandlePool.cpp:330.)
  return Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass


Train Epoch: 1 [3200/60000 (5%)]	Loss: 3987.237305
Train Epoch: 1 [6400/60000 (11%)]	Loss: 2932.038086
Train Epoch: 1 [9600/60000 (16%)]	Loss: 2598.258545
Train Epoch: 1 [12800/60000 (21%)]	Loss: 2138.980957
Train Epoch: 1 [16000/60000 (27%)]	Loss: 2184.477783
Train Epoch: 1 [19200/60000 (32%)]	Loss: 1862.265137
Train Epoch: 1 [22400/60000 (37%)]	Loss: 1778.291504
Train Epoch: 1 [25600/60000 (43%)]	Loss: 1979.121948
Train Epoch: 1 [28800/60000 (48%)]	Loss: 1934.015259
Train Epoch: 1 [32000/60000 (53%)]	Loss: 1656.599365
Train Epoch: 1 [35200/60000 (59%)]	Loss: 1622.959839
Train Epoch: 1 [38400/60000 (64%)]	Loss: 1671.950684
Train Epoch: 1 [41600/60000 (69%)]	Loss: 1779.286743
Train Epoch: 1 [44800/60000 (75%)]	Loss: 1544.889893
Train Epoch: 1 [48000/60000 (80%)]	Loss: 1636.269531
Train Epoch: 1 [51200/60000 (85%)]	Loss: 1611.844971
Train Epoch: 1 [54400/60000 (91%)]	Loss: 1579.648926
Train Epoch: 1 [57600/60000 (96%)]	Loss: 1591.298584
Epoch: 1 Average loss: 2167.016205
====> Test set:

KeyboardInterrupt: 

In [ ]:
import torch
import numpy as np
import matplotlib.pyplot as plt
# (CNF 및 ODEFunc 클래스는 이전에 정의된 셀에 존재해야 합니다.)

# ----------------------------------------------------
# 1. 설정값 및 모델 구조 정의
# ----------------------------------------------------
NUM_IMAGES_TO_GENERATE = 16 
INPUT_DIM = 28 * 28 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
TIME_T = 0.5 # 훈련 시 사용한 T 값과 일치해야 합니다.

# 모델 구조 정의 (가중치를 로드하기 위해 구조부터 생성)
# ⚠️ 주의: ODEFunc와 CNF 클래스는 이 셀보다 위에 정의되어 있어야 합니다.
try:
    ode_func = ODEFunc(dim=INPUT_DIM).to(device).float()
    model = CNF(ode_func, T=TIME_T).to(device).float()
except NameError:
    print("❌ 오류: ODEFunc 또는 CNF 클래스가 정의되지 않았습니다. 이전 셀을 실행해 주세요.")
    exit()

# ----------------------------------------------------
# 2. 학습된 Loss 550 모델 가중치 로드 (필수)
# ----------------------------------------------------
CHECKPOINT_PATH = './logs/ffjord_best_NORM.pthp' # 🟢 저장된 Loss 550대 모델 파일 경로

try:
    checkpoint = torch.load(CHECKPOINT_PATH)
    model.load_state_dict(checkpoint['model_state_dict'])
    best_loss = checkpoint.get('test_loss', 563.41) # test_loss가 있으면 가져오고, 없으면 563.41 사용
    print(f"✅ Loss {best_loss:.4f}의 모델 가중치 로드 성공.")
    
    # 모델 객체에 Loss 값을 저장하여 시각화에 사용
    model.current_loss = best_loss 
except FileNotFoundError:
    print(f"⚠️ 경고: 체크포인트 파일 '{CHECKPOINT_PATH}'을 찾을 수 없습니다. 초기화된 모델로 진행합니다.")
    best_loss = 780.0 # 초기 Loss 값으로 설정
except Exception as e:
    print(f"❌ 가중치 로드 중 알 수 없는 오류 발생: {e}. 초기화된 모델로 진행합니다.")
    best_loss = 780.0

# ----------------------------------------------------
# 3. 이미지 생성 및 시각화 함수 정의
# ----------------------------------------------------
import torch
import numpy as np
import matplotlib.pyplot as plt
# (CNF 및 ODEFunc 클래스는 이전에 정의되어 있어야 합니다.)

# ----------------------------------------------------
# 3. 이미지 생성 및 시각화 함수 정의 (Logit Transform 복원)
# ----------------------------------------------------
def generate_and_visualize(model, num_images, input_dim, device):
    
    model.eval() # 모델을 평가 모드로 설정

    # 1. Latent 벡터 생성 (z0 ~ N(0, I))
    z0 = torch.randn(num_images, input_dim).to(device).float() 
    z0.requires_grad_(True) # Trace Estimator 작동을 위해 필요
    # 2. logpz_0를 초기화 (Trace 항)
    zero_logpz = torch.zeros(z0.shape[0], 1).to(device).float() 
    
    # 3. forward pass 실행: x_gen은 정규화된 [-1, 1] 범위에 있음
    input_tuple = (z0, zero_logpz)
    x_gen, _ = model.forward(*input_tuple)
    x_denorm = torch.sigmoid(x_gen)  # logit 역변환
    x_denorm_clipped = torch.clamp(x_denorm, min=0.0, max=1.0)


    # 4. 이미지 형태로 복원 (바로 위에서 자른 변수 사용)
    x_images = x_denorm_clipped.detach().view(num_images, 28, 28).cpu().numpy()
    
    # 5. 시각화 및 파일 저장
    fig, axes = plt.subplots(4, 4, figsize=(6, 6))
    
    # Loss 값 표시 (로드된 Loss 사용)
    try:
        loss_display = f"Loss: {model.current_loss:.2f}"
    except AttributeError:
        # 모델에 current_loss가 없을 경우의 대체 값 사용 (새로운 학습 시)
        loss_display = "Loss: N/A (New Training)"
        
    plt.suptitle(f"FFJORD Generated Images ({loss_display})", fontsize=14)
    
    for i, ax in enumerate(axes.flat):
        if i < num_images:
            # vmin/vmax를 0과 1로 설정하여 올바른 흑백 명암비로 표시
            ax.imshow(x_images[i], cmap='gray', vmin=0, vmax=1)
            ax.axis('off')
        else:
            ax.axis('off')

    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    # 🟢 이미지 파일로 저장
    output_filename = "17ffjord_generated_images_no_logit_fix2_.png"
    plt.savefig(output_filename, dpi=300) 
    plt.close(fig)
    print(f"\n✅ 이미지 생성이 완료되어 '{output_filename}'으로 저장되었습니다.")

# ----------------------------------------------------
# 4. 함수 호출 예시 (메인 스크립트 마지막에 추가)
# ----------------------------------------------------
generate_and_visualize(model, NUM_IMAGES_TO_GENERATE, INPUT_DIM, device)

✅ Loss 1075.7429의 모델 가중치 로드 성공.

✅ 이미지 생성이 완료되어 '16ffjord_generated_images_no_logit_fix2_.png'으로 저장되었습니다.
